In [73]:
from datetime import datetime
import googlemaps
import config
import json

In [4]:
def _extract_directions(self, start_coords, end_coords, mode):
        """
        Extraction of Google Maps API data.

        Returns
        --------
        A json with trip direction information
        """

        # Call API
        directions = self._gmaps.directions(
            origin = start_coords,
            destination = end_coords,
            mode = mode,
            departure_time = self._trip_start
        )
        return directions

In [5]:
def get_coordinates(address, gmaps):
    """
    Returns
    -------
    Lat/Long corresponding to the supplied address.

    Parameters
    ----------
    address: [str] An address as you would enter it into Google Maps.
    gmaps: [googlemaps.Client object] Client object for current session.

    Examples
    --------
    >>> get_coordinates('One World Trade Center')
    {'lat': 40.7127431, 'lng': -74.0133795}

    >>> get_coordinates('476 5th Ave, New York, NY 10018')
    {'lat': 40.75318230000001, 'lng': -73.9822534}
    """
    geocoded_address = gmaps.geocode(address)
    geometry = geocoded_address[0]['geometry']['location']
    return geometry

In [ ]:
def extract_transform_directions(location_1, location_2, key):
    """
    Extracts and transforms data from the Google Maps API trip information
    between location_1 and location_2.

    Returns
    -------
    List of two Directions objects. One for a trip in each direction:

    location_1 -> location 2 AND
    location_2 -> location 1

    Parameters
    ----------
    location_1: [str] First location of this transit trip. An address as you
        would enter it into Google Maps.
    location_2: [str] Second location of this transit trip. An address as you
        would enter it into Google Maps.
    key: [str] Google API key.
    """
    gmaps = googlemaps.Client(key)
    gmaps = googlemaps.Client(config.api_key)

    # Get coordinates for two locations
    location_1_coords = get_coordinates(location_1, gmaps)
    location_2_coords = get_coordinates(location_2, gmaps)

    # Instantiate Direction objects for each trip (class handles extraction
    # and transformation of the data in the __init__ method)
    trip_1 = Directions(
        location_1_coords, location_2_coords, 'transit', gmaps
    )
    trip_2 = Directions(
        location_2_coords, location_1_coords, 'transit', gmaps
    )

    return [trip_1, trip_2]

In [8]:
# Insantiate gmaps client
gmaps = googlemaps.Client(config.api_key)

# Get coordinates for two locations
location_1_coords = get_coordinates('186 Scholes St, Brooklyn NY', gmaps)
location_2_coords = get_coordinates('110-48 72nd Ave, Flushing NY', gmaps)

In [48]:
# Call API
directions = gmaps.directions(
    origin = location_1_coords,
    destination = location_2_coords,
    mode = 'transit',
    departure_time = datetime.now()
)[0]

## Flow of Information

1. Calling gmaps.directions() gives back a JSON array with the directions data `directions`.
2. From `directions` we find the `legs`, which tells us information such as `arrival_time`, `departure_time`, `duration`, and the `steps` of the trip.
3. Parsing all of this will give a JSON for that particular trip.
4. Save JSON file to directory of directions files
5. Do the same for the opposite direction

In [ ]:
directions.keys() # Legs is the only useful one. Others are blank or gibberish

In [ ]:
directions['legs'][0].keys()

In [16]:
legs = directions['legs'][0]

dict_keys(['bounds', 'copyrights', 'legs', 'overview_polyline', 'summary', 'warnings', 'waypoint_order'])

In [85]:
trip_directions = {}

# Abbreviate the path
legs = directions['legs'][0]
arrival_time = legs['arrival_time']['value']     # Timestamp
departure_time = legs['departure_time']['value'] # Timestamp
duration = int(legs['duration']['value']/60)     # Minutes


# Abbreviate the next path
steps = legs['steps']
step_directions = []
for step in steps:
    
    # We'll only consider steps that involve bus or train.
    # Entries for walking don't habe key 'transit_details'
    if step.get('transit_details'):
        distance = step['distance']['value'] # Meters
        instrs = step['html_instructions']
        line_name = step['transit_details']['line']['short_name']
        step_directions.append({
            'distance': distance,
            'html_instructions': instrs,
            'line_name': line_name
        })

trip_directions = {
    'departure_time': departure_time,
    'arrival_time': arrival_time,
    'duration': duration,
    'steps': step_directions
}

In [86]:
trip_directions

{'departure_time': 1586492983,
 'arrival_time': 1586496469,
 'duration': 58,
 'steps': [{'distance': 11418,
   'html_instructions': 'Bus towards Jamaica 170 St',
   'line_name': 'Q54'},
  {'distance': 2564,
   'html_instructions': 'Subway towards World Trade Center',
   'line_name': 'E'}]}

In [90]:
# Export to a JSON
with open('directions.json', 'w') as f:
    json.dump(trip_directions, f)